In [ ]:
import getpass
import os
password = getpass.getpass()
#NOTE : !command --> pass the command to terminal, not python interpreter

In [ ]:
CUSTOM_MODEL_NAME ='my_ssd_mobnet'
PRETRAINED_MODEL_NAME = 'ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8'
PRETRAINED_MODEL_URL = 'http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz'
TF_RECORD_SCRIPT_NAME = 'generate_tfrecord.py'
LABEL_MAP_NAME = 'label_map.pbtxt'

In [ ]:
CUSTOM_MODEL_NAME

In [ ]:
paths ={
    'WORKSPACE_PATH':os.path.join('RicefieldTF','Tensorflow','workspace'),
    'SCRIPTS_PATH':os.path.join('RicefieldTF','Tensorflow','scripts'),
    'APIMODEL_PATH':os.path.join('RicefieldTF','Tensorflow','models'),
    'ANNOTATION_PATH':os.path.join('RicefieldTF','Tensorflow','workspace','annotations'),
    'IMAGE_PATH':os.path.join('RicefieldTF','Tensorflow','workspace','images'),
    'MODEL_PATH':os.path.join('RicefieldTF','Tensorflow','workspace','models'),
    'PRETRAINED_MODEL_PATH':os.path.join('RicefieldTF','Tensorflow','workspace','pre-trained-models'),
    'CHECKPOINT_PATH':os.path.join('RicefieldTF','Tensorflow','workspace','models',CUSTOM_MODEL_NAME),
    'OUTPUT_PATH':os.path.join('RicefieldTF','Tensorflow','workspace','models',CUSTOM_MODEL_NAME,'export'),
    'TFJS_PATH':os.path.join('RicefieldTF','Tensorflow','workspace','models',CUSTOM_MODEL_NAME,'tfjsexport'),
    'TFLITE_PATH':os.path.join('RicefieldTF','Tensorflow','workspace','models',CUSTOM_MODEL_NAME,'tfjliteport'),
    'PROTOC_PATH':os.path.join('RicefieldTF','Tensorflow','protoc')
}

In [ ]:
files={
    'PIPELINE_CONFIG':os.path.join('RicefieldTF','Tensorflow','workspace','models',CUSTOM_MODEL_NAME,'pipeline.config'),
    'TF_RECORD_SCRIPT':os.path.join(paths['SCRIPTS_PATH'], TF_RECORD_SCRIPT_NAME),
    'LABELMAP':os.path.join(paths['ANNOTATION_PATH'],LABEL_MAP_NAME)    
}

In [ ]:
for path in paths.values():
    if not os.path.exists(path):
        if os.name == 'posix':
            !mkdir -p {path}
        if os.name =='nt':
            !mkdir {path}
print(os.name)

## Download TF Models and Install TFOD

In [ ]:
if os.name == 'nt':
    !pip install wget
    import wget

In [ ]:
#clone tf model repo
if not os.path.exists(os.path.join(paths['APIMODEL_PATH'],'research','object_detection')):
    !git clone https://github.com/tensorflow/models {paths['APIMODEL_PATH']}

In [ ]:
# Install Tensorflow Object Detection 
#linux
if os.name=='posix':  
    #install protocol buffers
    #run this command = "sudo apt-get install protobuf-compiler"
    !cd RicefieldTF/Tensorflow/models/research && protoc object_detection/protos/*.proto --python_out=. && cp object_detection/packages/tf2/setup.py . && python -m pip install . 
    
#windows
if os.name=='nt':
    url="https://github.com/protocolbuffers/protobuf/releases/download/v3.15.6/protoc-3.15.6-win64.zip"
    wget.download(url)
    !move protoc-3.15.6-win64.zip {paths['PROTOC_PATH']}
    !cd {paths['PROTOC_PATH']} && tar -xf protoc-3.15.6-win64.zip
    os.environ['PATH'] += os.pathsep + os.path.abspath(os.path.join(paths['PROTOC_PATH'], 'bin'))   
    !cd RicefieldTF/Tensorflow/models/research && protoc object_detection/protos/*.proto --python_out=. && copy object_detection\\packages\\tf2\\setup.py setup.py && python setup.py build && python setup.py install
    !cd RicefieldTF/Tensorflow/models/research/slim && pip install -e .

In [ ]:
VERIFICATION_SCRIPT = os.path.join(paths['APIMODEL_PATH'], 'research', 'object_detection', 'builders', 'model_builder_tf2_test.py')
# Verify Installation
# DO NOT FORGET pip install tensorflow --upgrade , pip install matplotlib==3.2 if needed before running this
# Get this in the end  --> OK (skipped=1)
!python {VERIFICATION_SCRIPT}

In [17]:
import object_detection

In [18]:
!pip list

Package                       Version
----------------------------- -------------------
absl-py                       0.12.0
apache-beam                   2.31.0
astunparse                    1.6.3
attrs                         21.2.0
avro-python3                  1.9.2.1
backcall                      0.2.0
cachetools                    4.2.2
certifi                       2021.5.30
cffi                          1.14.6
chardet                       4.0.0
contextlib2                   21.6.0
crcmod                        1.7
cycler                        0.10.0
Cython                        0.29.23
debugpy                       1.3.0
decorator                     5.0.9
dill                          0.3.1.1
dm-tree                       0.1.6
docopt                        0.6.2
fastavro                      1.4.2
flatbuffers                   1.12
future                        0.18.2
gast                          0.4.0
gin-config                    0.4.0
google-api-core               1.31

In [19]:
#SINCE IM USING A LAPTOP WITHOUT CUDA AND CUDNN, IM JUST GOING TO SKIP THIS. GOOD LUCK CPU

In [22]:
#get pretrained model
#you know what im just gonna assume this is linux ok can? let me know if you want the windows version
if os.name =='posix':
    !wget {PRETRAINED_MODEL_URL}
    !mv {PRETRAINED_MODEL_NAME+'.tar.gz'} {paths['PRETRAINED_MODEL_PATH']}
    !cd {paths['PRETRAINED_MODEL_PATH']} && tar -zxvf {PRETRAINED_MODEL_NAME+'.tar.gz'}

--2021-07-11 15:14:38--  http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz
Resolving download.tensorflow.org (download.tensorflow.org)... 142.251.10.128, 2404:6800:4003:c03::80
Connecting to download.tensorflow.org (download.tensorflow.org)|142.251.10.128|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 20515344 (20M) [application/x-tar]
Saving to: ‘ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz.1’

ssd_mobilenet_v2_fp 100%[===================>]  19.56M  12.8MB/s    in 1.5s    

2021-07-11 15:14:40 (12.8 MB/s) - ‘ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz.1’ saved [20515344/20515344]

ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/
ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/checkpoint/
ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/checkpoint/ckpt-0.data-00000-of-00001
ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/checkpoint/checkpoint
ssd_mobilenet_v2_fpnlite_320x320_coco17_

In [23]:
#habis ini nyoba pake in label apa ya?

In [26]:
#tflite pake ini: pip3 install --index-url https://google-coral.github.io/py-repo/ tflite_runtime
!pip3 install --index-url https://google-coral.github.io/py-repo/ tflite_runtime

Looking in indexes: https://google-coral.github.io/py-repo/
     |████████████████████████████████| 1.2 MB 2.2 MB/s eta 0:00:01


In [27]:
import tflite_runtime.interpreter as tflite